In [36]:
from datetime import datetime
import pandas as pd
import random
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import *
from time import sleep

service = Service('geckodriver.exe')
options = webdriver.FirefoxOptions()
# options.add_argument('-headless')
base_url = 'https://www.beckershospitalreview.com'
driver = webdriver.Firefox(service=service, options=options)
wait = WebDriverWait(driver, 10)

pages = range(0, 20, 20)
urls = []
for page in pages:
    url = f'https://www.beckershospitalreview.com/?start={page}'
    urls.append(url)

article_list = []
for url in urls:
    driver.get(url)
    body = driver.page_source
    html = HTML(html=body)

    articles = html.find('.article')

    for article in articles:
        try:
            title = article.find('.article-title', first=True).text
        except:
            None
        try:
            link = base_url + \
                article.find('.article-title', first=True).links.pop()
        except:
            None
        try:
            date = article.find('.article-date > time', first=True)
            date = date.attrs['datetime']
            date = date.split('T')[0]
        except:
            None
        try:
            channel = link.split('/')[3]
        except:
            None

        data = {
            'Date': date,
            'Channel': channel,
            'Title': title,
            'Link': link
        }

        article_list.append(data)

        n = random.randint(1, 5)
        sleep(n)

driver.close()

df = pd.DataFrame(article_list)
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
df['Source'] = 'Becker\'s Hospital Review'
df = df[['Date', 'Source', 'Channel', 'Title', 'Link']]
df.drop_duplicates(inplace=True)
datestamp = datetime.today().strftime('%Y%m%dT%H%M')
df.to_csv(f'Beckers_Hospital_Review_Articles_{datestamp}.csv', index=None)


Public health, preventive medicine is the specialty with the lowest burnout: 3 survey findings
Health spending grew 3.4% in 2021, report finds: 7 things to know
Cerner launches clinical care council to improve inclusive care
New York backs off enforcement of booster mandate after plea from hospitals
Congress passes clinician mental health bill: 5 things to know
Telehealth has some fierce advocates: 5 moves to keep it from fading away
What investments are healthcare CFOs making in 2022? 6 survey findings
Hospital M&A: 6 recent deals
Who is getting boosters? 3 notes per CDC data
5 stats on burnout among infectious disease physicians
Healthcare billing fraud: 9 recent cases
18 data points that make EHRs attractive to hackers
10 places in US with the highest, lowest nursing home facility rates
What 5 studies say about telehealth's potential to shrink health disparities
Hospital rebrands: 6 recent name changes
24 recent hospital, health system executive moves
The 55 hospitals penalized by M